In [1]:
import math
from time import time
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
import pandas as pd
import sklearn as sk
import sklearn.ensemble as ske
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [2]:
# Removes a warning in sklearn that will be fixed during an update mid 2018
import warnings

if __name__ == '__main__':
    warnings.filterwarnings(action='ignore', category=DeprecationWarning)
    le = sk.preprocessing.LabelEncoder()
    le.fit([1, 2, 2, 6])
    le.transform([1, 1, 2, 6])
    le.inverse_transform([0, 0, 1, 2])

In [3]:
df = pd.read_csv('C:/GitHub/kaggle/titanic/data/train.csv', sep=',', header=0)
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
def substrings_in_string(whole, subs): 
    
    for x in subs:
        if x in str(whole): 
            return x
        
    return 'Unknown'

In [5]:
def extract_title(full_name):
    
    full_name  = str(full_name)
    
    x = full_name.split(", ")
    x = x[1]
    x = x.split('.')
    x = x[0]
    
    return x

In [6]:
def simplify_titles(x):
    
    title=x['Title']
    
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col', 'Sir']:
        return 'Mr'
    elif title in ['the Countess', 'Mme', 'Dona', 'Lady']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title == 'Dr':
        if x['Sex'] == 0:
            return 'Mrs'
        else:
            return 'Mr'
    else:
        return title
    

In [7]:
def preprocess_dataframe(df, prediction_data=False, print_info=False):
    """
    Description:
    
    Performs preprocessing on the titanic data
    
    PassengerId - Id (Only available on training data)
    Survived    - Survived  (0 = No; 1 = Yes)
    Pclass      - Passenger Class  (1 = 1st; 2 = 2nd; 3 = 3rd)
    Name        - Name
    Sex         - Sex
    Age         - Age
    Sibsp       - Number of Siblings/Spouses Aboard
    Parch       - Number of Parents/Children Aboard
    Ticket      - Ticket Number
    Fare        - Passenger Fare (British pound)
    Cabin       - Cabin code
    Embarked    - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
    
    Arguments:
    df -- Dataset, Pandas DataFrame
    
    Returns:
    df -- Dataset, Pandas DataFrame
    """

    # Age has missing values which is replaced with average
    # Might also consider dividing age into classes of age brackets
    df['Age'].fillna((df['Age'].mean()), inplace = True)
    df['Fare'].fillna((df['Fare'].mean()), inplace = True)
    
    # Encode sex into binary (0 = male, 1 = female)
    df['Sex'] = df['Sex'].map({'female': 1, 'male': 0})
    
    # Turning Cabin number into Deck
    cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
    df['Deck' ]= df['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
    
    # Calculate family size
    df['FamilySize'] = df['SibSp'] + df['Parch']
    
    # Age brackets
    df['AgeBracket'] = df['Age'].apply(lambda x: 'Child' if x <= 10 else ('Adult' if x < 50 else 'Senior'))

    """
    # Extract and process titles
    df['Title']= df['Name'].map(lambda x: extract_title(x))
    if not prediction_data:
        unique_titles = df['Title'].unique()
        survival_by_title = df.groupby('Title').mean()['Survived']
        
    df['Title']=df.apply(simplify_titles, axis=1)
    if not prediction_data:
        unique_titles_simplified = df['Title'].unique()
        survival_by_title_simplified = df.groupby('Title').mean()['Survived']
    """
    

    # Gather info on the significance of these classes for survival
    # Class was, as expected, significance for surival with the rates (1st - 63%, 2nd - 47%, 3rd - 24%)
    # Embarked was suprisingly significant, C - Cherbourg had 55% surivial rate when the mean was just 38%
    if not prediction_data:
        survival_by_plcass = df.groupby('Pclass').mean()['Survived']
        survival_by_deck = df.groupby('Deck').mean()['Survived']
        survival_by_embark = df.groupby('Embarked').mean()['Survived']
        survival_by_familysize = df.groupby('FamilySize').mean()['Survived']
        survival_by_agebracket = df.groupby('AgeBracket').mean()['Survived']
    
    # Split classes with one hot encoding
    # Pclass   - splits into (1 = Pclass_1, 2 = Pclass_2, 3 = Pclass_3)
    # Embarked - splits into (C = Embarked_C, Q = Embarked_Q, S = Embarked_S)
    # Deck - splits into decks with letters
    df = pd.get_dummies(df, columns = ['Pclass', 'Embarked', 'Deck', 'AgeBracket']) #, 'Title'])

    # Drop columns with data deemed not relevant for learning
    # Name     - Gender already has its' own column. Only thing that might be interesting here is the title
    # Ticket   - Ticket does not really say much, price and class are already included which says the most
    # Cabin    - Replaced by Deck
    df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Age'], axis=1) #, 'SibSp', 'Parch'], axis=1)
    
    if prediction_data:
        m = df.shape[0]
        df['Deck_T'] = pd.Series(np.zeros(m, dtype=int), index=df.index)
    
    # Normalize the data
    norm_vals = ['Fare', 'FamilySize', 'SibSp', 'Parch'] #, 'Age']
    df[norm_vals] = (df[norm_vals] - df[norm_vals].min())/(df[norm_vals].max() - df[norm_vals].min())
    
    if print_info:
        if not prediction_data:
            print('--------------------------------------------------------------------------------------')
            print('SURVIVAL RATE')
            print('--------------------------------------------------------------------------------------')
            print('Overall survival rate: ' + str(df['Survived'].mean()))
            print()
            print(survival_by_plcass)
            print()
            print(survival_by_embark)
            print()
            print(survival_by_deck)
            print()
            #print(survival_by_title)
            print()
            #print(survival_by_title_simplified)
            print()
            print(survival_by_familysize)
            print()
            print(survival_by_agebracket)
        """
        print('--------------------------------------------------------------------------------------')
        print('TITLES')
        print('--------------------------------------------------------------------------------------')
        print('All titels: ')
        print(unique_titles)
        print()
        print('Simplied titels: ')
        print(unique_titles_simplified)
        """
        print('--------------------------------------------------------------------------------------')
        print('SUMS')
        print('--------------------------------------------------------------------------------------')
        print(df.sum())
        print('--------------------------------------------------------------------------------------')
        print('DATA INFO')
        print('--------------------------------------------------------------------------------------')
        print(df.info())
        print('--------------------------------------------------------------------------------------')
        print('MISSING VALUES')
        print('--------------------------------------------------------------------------------------')
        print(df.isnull().sum())
        print('--------------------------------------------------------------------------------------')
        print('CORRELATIONS')
        print('--------------------------------------------------------------------------------------')
        print(df.corr())
        print('--------------------------------------------------------------------------------------')

    return df

In [8]:
def split_data(df):
    """
    Description:
    Splits the data into test/training set for simple validation

    Arguments:
    df -- Dataset, pandas dataframe
        
    Returns:
    train -- Training samples, pandas dataframe
    test -- Test samples, pandas dataframe
    """

    df_train = df.sample(frac = 0.8, random_state = 42)
    df_test = df.drop(df_train.index)  
    
    X_train = df_train.drop(['Survived'], axis=1).values
    y_train = df_train['Survived'].values
    
    X_test = df_test.drop(['Survived'], axis=1).values
    y_test = df_test['Survived'].values
    
    return X_train, y_train, X_test, y_test

In [9]:
df_processed = preprocess_dataframe(df, False, False)

X_train, y_train, X_test, y_test = split_data(df_processed)

print ("X_train shape: " + str(X_train.shape))
print ("y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("y_test shape: " + str(y_test.shape))

df_processed.head(5)

X_train shape: (713, 23)
y_train shape: (713,)
X_test shape: (178, 23)
y_test shape: (178,)


,Survived,Sex,SibSp,Parch,Fare,FamilySize,Pclass_1,Pclass_2,Pclass_3,Embarked_C,...,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_Unknown,AgeBracket_Adult,AgeBracket_Child,AgeBracket_Senior
0,0,0,0.125,0.0,0.014151,0.1,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
1,1,1,0.125,0.0,0.139136,0.1,1,0,0,1,...,1,0,0,0,0,0,0,1,0,0
2,1,1,0.000,0.0,0.015469,0.0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
3,1,1,0.125,0.0,0.103644,0.1,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
4,0,0,0.000,0.0,0.015713,0.0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0


In [10]:
def decision_tree_clf(X_train, y_train, X_test, y_test):
    """
    Description:
    Builds and trains a decision tree classifier

    Arguments:
    X_train -- Training features,numpy matrix (m, 11)
    y_train -- Training features,numpy matrix (m, )
    X_test -- Test features,numpy matrix (m, 11)
    y_test -- Test features,numpy matrix (m, )
        
    Returns:
    dt_clf -- Classifier, sklearn DecisionTreeClassifier
    """
    
    dt_clf = sk.tree.DecisionTreeClassifier(max_depth=20)
    dt_clf.fit (X_train, y_train)
    print(dt_clf.score (X_test, y_test))
    
    return dt_clf

In [11]:
dt_clf = decision_tree_clf(X_train, y_train, X_test, y_test)

0.8089887640449438


In [12]:
def random_forest_clf(X_train, y_train, X_test, y_test):
    """
    Description:
    Builds and trains a random forest classifier

    Arguments:
    X_train -- Training features,numpy matrix (m, 11)
    y_train -- Training features,numpy matrix (m, )
    X_test -- Test features,numpy matrix (m, 11)
    y_test -- Test features,numpy matrix (m, )
        
    Returns:
    rf_clf -- Classifier, sklearn RandomForestClassifier
    """
    
    rf_clf = ske.RandomForestClassifier(n_estimators=50)
    rf_clf.fit (X_train, y_train)
    print(rf_clf.score (X_test, y_test))
    
    return rf_clf

In [13]:
rf_clf = random_forest_clf(X_train, y_train, X_test, y_test)

0.8033707865168539


In [14]:
def gradient_boosting_clf(X_train, y_train, X_test, y_test):
    """
    Description:
    Builds and trains a gradient boosting classifier

    Arguments:
    X_train -- Training features,numpy matrix (m, 11)
    y_train -- Training features,numpy matrix (m, )
    X_test -- Test features,numpy matrix (m, 11)
    y_test -- Test features,numpy matrix (m, )
        
    Returns:
    gb_clf -- Classifier, sklearn GradiantBoostingClassifier
    """
    
    gb_clf = ske.GradientBoostingClassifier(n_estimators=50)
    gb_clf.fit (X_train, y_train)
    print(gb_clf.score (X_test, y_test))
    
    return gb_clf

In [15]:
gb_clf = gradient_boosting_clf(X_train, y_train, X_test, y_test) 

0.8370786516853933


In [16]:
def logistic_regression_clf(X_train, y_train, X_test, y_test):
    """
    Description:
    Builds and trains a logistic regression classifier

    Arguments:
    X_train -- Training features,numpy matrix (m, 11)
    y_train -- Training features,numpy matrix (m, )
    X_test -- Test features,numpy matrix (m, 11)
    y_test -- Test features,numpy matrix (m, )
        
    Returns:
    rf_clf -- Classifier, sklearn LogisticRegression
    """
    
    lr_clf = LogisticRegression()
    lr_clf.fit (X_train, y_train)
    print(lr_clf.score (X_test, y_test))
    
    return lr_clf

In [17]:
lr_clf = logistic_regression_clf(X_train, y_train, X_test, y_test)

0.8202247191011236


In [18]:
def support_vector_machine_clf(X_train, y_train, X_test, y_test):
    """
    Description:
    Builds and trains a support vector machine classifier

    Arguments:
    X_train -- Training features,numpy matrix (m, 11)
    y_train -- Training features,numpy matrix (m, )
    X_test -- Test features,numpy matrix (m, 11)
    y_test -- Test features,numpy matrix (m, )
        
    Returns:
    svm_clf -- Classifier, sklearn SVC
    """
    
    svm_clf = sk.svm.SVC(probability=True)
    svm_clf.fit (X_train, y_train)
    print(svm_clf.score (X_test, y_test))
    
    return svm_clf

In [19]:
svm_clf = support_vector_machine_clf(X_train, y_train, X_test, y_test) 

0.8089887640449438


In [20]:
def naive_bayes_clf(X_train, y_train, X_test, y_test):
    """
    Description:
    Builds and trains a gaussian naive bayes classifier

    Arguments:
    X_train -- Training features,numpy matrix (m, 11)
    y_train -- Training features,numpy matrix (m, )
    X_test -- Test features,numpy matrix (m, 11)
    y_test -- Test features,numpy matrix (m, )
        
    Returns:
    svm_clf -- Classifier, sklearn GaussianNB
    """
    
    nb_clf = GaussianNB()
    nb_clf.fit (X_train, y_train)
    print(nb_clf.score (X_test, y_test))
    
    return nb_clf

In [21]:
nb_clf = naive_bayes_clf(X_train, y_train, X_test, y_test)

0.7191011235955056


In [22]:
def k_neighbors_clf(X_train, y_train, X_test, y_test):
    """
    Description:
    Builds and trains a k-nearest neighbors classifier

    Arguments:
    X_train -- Training features,numpy matrix (m, 11)
    y_train -- Training features,numpy matrix (m, )
    X_test -- Test features,numpy matrix (m, 11)
    y_test -- Test features,numpy matrix (m, )
        
    Returns:
    svm_clf -- Classifier, sklearn KNeighborsClassifier
    """
    
    knn_clf = KNeighborsClassifier(n_neighbors=6)
    knn_clf.fit (X_train, y_train)
    print(knn_clf.score (X_test, y_test))
    
    return knn_clf

In [23]:
knn_clf = k_neighbors_clf(X_train, y_train, X_test, y_test)

0.8033707865168539


In [38]:
def ensamble_voting_clf(clfs, X_train, y_train, X_test, y_test, cv=20, score_individually=False):
    """
    Description:
    Builds and trains an ensamble of classifiers which them vote together

    Arguments:
    clfs -- Classifiers with labels, List Tuple(String, clf)
    
    Returns:
    e_clf -- Classifier, sklearn VotingClassifier
    """
    
    e_clf = ske.VotingClassifier(estimators=clfs, voting='hard') # Hard voting where majority rules
    e_clf.fit (X_train, y_train)
    
    if score_individually:
        for label, clf in clfs:
            scores = cross_val_score(clf, X_test, y_test, cv=cv, scoring='accuracy')
            print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))    
    
    scores = cross_val_score(e_clf, X_test, y_test, cv=cv, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'Voting Ensamble')) 
    
    return e_clf    

In [25]:
clfs = ([('Decision Tree', dt_clf), ('Random Forest', rf_clf), ('Gradiant Boosting', gb_clf), 
         ('Logistic Regression', lr_clf), ('SVM', svm_clf), ('Naive Bayes', nb_clf), ('K-Nearest', knn_clf)])

e_clf = ensamble_voting_clf(clfs, X_train, y_train, X_test, y_test, score_individually=True)

Accuracy: 0.80 (+/- 0.09) [Decision Tree]
Accuracy: 0.82 (+/- 0.08) [Random Forest]
Accuracy: 0.81 (+/- 0.11) [Gradiant Boosting]
Accuracy: 0.84 (+/- 0.12) [Logistic Regression]
Accuracy: 0.81 (+/- 0.10) [SVM]
Accuracy: 0.45 (+/- 0.13) [Naive Bayes]
Accuracy: 0.79 (+/- 0.11) [K-Nearest]
Accuracy: 0.83 (+/- 0.09) [Voting Ensamble]


In [26]:
def predict(df, clf, export_path):
    """
    Description:
    Makes predictions X -> y and exports to csv

    Arguments:
    df -- Data to predict from, pandas DataFrame
    clf -- classifier, Classifier, sklearn classifier object
    export_path -- Path and name of file, String
        
    Returns:
    df_pred -- prediction, pandas DataFrame
    """
    
    # Extract Ids
    y1 = df['PassengerId'].values
    
    # Make predictions
    df_process = preprocess_dataframe(df, prediction_data=True, print_info=False) 
    X = df_process.values
    y2 = clf.predict(X)
    
    # Combine ids and predictions
    y = np.column_stack((y1, y2))
    
    # Restore pandas df
    df_pred = pd.DataFrame(y)
    df_pred.columns = ["PassengerId", "Survived"]
    
    # Export
    df_pred.to_csv(export_path, sep=',', index=False)
    
    return df_pred

In [27]:
df_pred = pd.read_csv('C:/GitHub/kaggle/titanic/data/test.csv', sep=',', header=0)
df_pred = predict(df_pred, rf_clf, 'C:/GitHub/kaggle/titanic/predictions/predictions_random_forest.csv')
df_pred.head(5)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [28]:
def report(results, n_top=1):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [29]:
def random_search_hyperparameters(clf, hyper_param, X_train, y_train, X_test, y_test, print_result=False):
    
    # run randomized search
    n_iter_search = 10
    rnd_clf = RandomizedSearchCV(clf, param_distributions=hyper_param, n_iter=n_iter_search)

    start = time()
    rnd_clf.fit(X_train, y_train)
    
    if print_result:
        print("RandomizedSearchCV took %.2f seconds for %d candidates"
              " parameter settings." % ((time() - start), n_iter_search))
        report(rnd_clf.cv_results_)
        y_true, y_pred = y_test, clf.predict(X_test)
        print(classification_report(y_true, y_pred))
       
    return rnd_clf  

In [30]:
def grid_search_hyperparameters(clf, hyper_param, X_train, y_train, X_test, y_test, print_result=False):
    
    # run grid search
    grid_clf = GridSearchCV(clf, param_grid=hyper_param)
    start = time()
    grid_clf.fit(X_train, y_train)
    
    if print_result:
        report(grid_clf.cv_results_)
        y_true, y_pred = y_test, clf.predict(X_test)
        print(classification_report(y_true, y_pred))
    
    return grid_clf  

In [31]:
# specify parameters and distributions to sample from
hyper_param = {"max_depth": [3, None],
               "max_features": stats.randint(3, X_train.shape[1]),
               "min_samples_split": stats.randint(2, 11),
               "min_samples_leaf": stats.randint(1, 11),
               "bootstrap": [True, False],
               "criterion": ["gini", "entropy"]}

rnd_rf_clf = random_search_hyperparameters(rf_clf, hyper_param, X_train, y_train, X_test, y_test)

In [32]:
# use a full grid over all parameters
hyper_param = {"max_depth": [3, None],
               "max_features": [3, 10, X_train.shape[1]],
               "min_samples_split": [2, 3, 10],
               "min_samples_leaf": [1, 3, 10],
               "bootstrap": [True, False],
               "criterion": ["gini", "entropy"]}

rf_grid_clf = grid_search_hyperparameters(rf_clf, hyper_param, X_train, y_train, X_test, y_test, True)

Model with rank: 1
Mean validation score: 0.827 (std: 0.013)
Parameters: {'max_features': 23, 'criterion': 'entropy', 'min_samples_leaf': 3, 'bootstrap': True, 'max_depth': None, 'min_samples_split': 2}

             precision    recall  f1-score   support

          0       0.86      0.82      0.84       113
          1       0.71      0.77      0.74        65

avg / total       0.81      0.80      0.80       178



In [33]:
# specify parameters and distributions to sample from
hyper_param = {'C': stats.randint(10, 10000),
               'gamma' : stats.uniform(0.001, 1),
               'kernel': ['rbf']}

svm_rnd_clf = random_search_hyperparameters(svm_clf, hyper_param, X_train, y_train, X_test, y_test)

In [34]:
# use a full grid over all parameters
hyper_param = {'C': [10, 100, 1000, 10000], 
               'gamma' : [0.001, 0.01, 0.1, 1], 
               'kernel': ['rbf']}

svm_grid_clf = grid_search_hyperparameters(svm_clf, hyper_param, X_train, y_train, X_test, y_test, True)

Model with rank: 1
Mean validation score: 0.820 (std: 0.002)
Parameters: {'kernel': 'rbf', 'gamma': 0.01, 'C': 1000}

Model with rank: 1
Mean validation score: 0.820 (std: 0.006)
Parameters: {'kernel': 'rbf', 'gamma': 0.01, 'C': 10000}

             precision    recall  f1-score   support

          0       0.84      0.87      0.85       113
          1       0.75      0.71      0.73        65

avg / total       0.81      0.81      0.81       178



In [35]:
# specify parameters and distributions to sample from
hyper_param = {'penalty': ['l1','l2'], 
               'C': stats.uniform(0.001,1000)}

lr_rnd_clf = random_search_hyperparameters(lr_clf, hyper_param, X_train, y_train, X_test, y_test)

In [36]:
# use a full grid over all parameters
hyper_param = {'penalty': ['l1','l2'], 
               'C': [0.001,0.01,0.1,1,10,100,1000]}

lr_grid_clf = grid_search_hyperparameters(lr_clf, hyper_param, X_train, y_train, X_test, y_test, True)

Model with rank: 1
Mean validation score: 0.808 (std: 0.003)
Parameters: {'penalty': 'l2', 'C': 0.1}

             precision    recall  f1-score   support

          0       0.87      0.84      0.86       113
          1       0.74      0.78      0.76        65

avg / total       0.82      0.82      0.82       178



In [39]:
clfs = ([('Grid Random Forest', rf_grid_clf), ('Grid SVM', svm_grid_clf), ('Grid Logistic Regression', lr_grid_clf)])

e_grid_clf = ensamble_voting_clf(clfs, X_train, y_train, X_test, y_test, score_individually=False, cv=5)

Accuracy: 0.82 (+/- 0.02) [Voting Ensamble]


In [41]:
df_pred = pd.read_csv('C:/GitHub/kaggle/titanic/data/test.csv', sep=',', header=0)
df_pred = predict(df_pred, e_grid_clf, 'C:/GitHub/kaggle/titanic/predictions/predictions_tuned_voting_ensamble.csv')
df_pred.head(5)

,PassengerId,Survived
0,892,0
1,893,1
2,894,1
3,895,0
4,896,1
